In [1]:
import generative_model_score
score_model = generative_model_score.GenerativeModelScore()
score_model.lazy_mode(True)
import dataset
import TorchKmeans
train_loader = dataset.get_cifar1_dataset(2048, 32, shuffle=False)
data_cl_loader, numdata_in_cl = TorchKmeans.clusterset_with_gm_load_or_make(score_model, train_loader)
#score_model.load_or_make(train_loader)

import torch


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


Files already downloaded and verified


  0%|          | 0/782 [00:00<?, ?it/s]

generate real images info


100%|██████████| 782/782 [02:17<00:00,  5.71it/s]


K-means for the Euclidean metric with 50,000 points in dimension 2,048, K = 100:
Timing for 100 iterations: 1.87001s = 100 x 0.01870s

make and save pickle file : %s ../inception_model_info/9aa4ddb3bd06dd3539a303459af7736c.pickle_cls_info.pickle


In [10]:
device = 'cuda:0'
n_epoch = 1e+6

In [43]:
import torch.nn as nn
ndf = 64
ngf = 64
nz = 100
nc = 3
ngpu = 1

class Assigner(nn.Module):
    def __init__(self, ngpu, K):
        super(Assigner, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Flatten(),
            #  -- original code --
            # state size. (ndf*8) x 4 x 4
            #nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            #nn.Conv2d(ndf * 8, 1, 2, 1, 0, bias=False),
            # state size. 1x1x1
            #nn.Sigmoid()
            nn.Linear(2048, K)
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)

assign = Assigner(ngpu, 100).to(device)

In [44]:
criterion = torch.nn.CrossEntropyLoss()
assign_optim = torch.optim.Adam(assign.parameters(), lr=1e-4)

In [45]:
epoch = 0
while epoch < n_epoch : 
    for i, (data, cluster) in enumerate(data_cl_loader) :         
        batch_size = data.size(0)
        image_cuda = data.to(device)
        cluster_cuda = cluster.to(device)
        predict_cluster = assign(image_cuda).view(batch_size, -1)
        loss = criterion(predict_cluster, cluster_cuda)
        hit = torch.argmax(predict_cluster, dim=1) == cluster_cuda
                
        if i >= 20 : break
        
        assign_optim.zero_grad()
        loss.backward()
        assign_optim.step()
        
    acc = torch.sum(hit).float()/len(hit)
    print(epoch, acc.item(), loss.item())
    epoch += 1

0 0.1025390625 3.883622884750366
1 0.162109375 3.589569568634033
2 0.1865234375 3.3730077743530273
3 0.19970703125 3.2602040767669678
4 0.228515625 3.111415147781372
5 0.25439453125 3.010782241821289
6 0.29541015625 2.846712827682495
7 0.3046875 2.785303831100464
8 0.34130859375 2.638354778289795
9 0.3583984375 2.55979585647583
10 0.40234375 2.385891914367676
11 0.3994140625 2.393885374069214
12 0.45654296875 2.2261126041412354
13 0.44287109375 2.193819999694824
14 0.48583984375 2.0398716926574707
15 0.50146484375 2.0078928470611572
16 0.53173828125 1.9140528440475464
17 0.56103515625 1.821030616760254
18 0.5810546875 1.7712167501449585
19 0.61328125 1.6462348699569702
20 0.6298828125 1.6269111633300781
21 0.6845703125 1.4747446775436401
22 0.6943359375 1.4289196729660034
23 0.71826171875 1.3497103452682495
24 0.74609375 1.2601025104522705
25 0.75390625 1.2063143253326416
26 0.77490234375 1.17573082447052
27 0.78759765625 1.0998814105987549
28 0.81640625 1.0226222276687622
29 0.8442382

KeyboardInterrupt: 

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()


    parser.add_argument('--device', type=str, default='cuda')
    parser.add_argument('--epochs', type=int, default=100)
    parser.add_argument('--batch_size', type=int, default=64)
    parser.add_argument('--img_size', type=int, default=32)
    parser.add_argument('--save_image_interval', type=int, default=5)
    parser.add_argument('--loss_calculation_interval', type=int, default=5)
    parser.add_argument('--latent_dim', type=int, default=10)
    parser.add_argument('--n_iter', type=int, default=3)
    parser.add_argument('--project_name', type=str, default='AAE')
    parser.add_argument('--dataset', type=str, default='', choices=['LSUN_dining_room', 'LSUN_classroom', 'LSUN_conference', 'LSUN_churches',
                                                                    'FFHQ', 'CelebA', 'cifar10', 'mnist', 'mnist_fashion', 'emnist'])

    parser.add_argument('--model_name', type=str, default='', choices=['vanilla', 'yeop_loss', 
                                                                       'yeop_n_iter', 'mod_var', 'mod2_var', 'gme', 'latent_mapping', 'gme_inference', 'direct'])

    parser.add_argument('--lr', type=float, default=1e-4)
    parser.add_argument('--run_test', type=bool, default=False)

    args = parser.parse_args()

    main(args)